<a href="https://colab.research.google.com/github/hailong18102002/computer-vision/blob/main/bilinear_interpolation_Alleysons_malva_2004.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

**import lib**

In [ ]:
import cv2
import math
import numpy as np
from matplotlib import pyplot as plt
import os
from PIL import Image
from google.colab.patches import cv2_imshow
from sklearn.preprocessing import MinMaxScaler
import scipy.signal as signal
import matplotlib.image as mpimg
import matplotlib.cm as cm
import skimage.io
import skimage.metrics
import tensorflow as tf


**malva-2004**

In [ ]:
def make_cfa(img):
    cfa = np.zeros_like(img[:,:,0])
    y, x = np.mgrid[0:cfa.shape[0], 0:cfa.shape[1]]
    Rloc = (np.mod(x, 2) == 0) & (np.mod(y, 2) == 0)
    Gloc = np.mod(x+y, 2) == 1
    Bloc = (np.mod(x, 2) == 1) & (np.mod(y, 2) == 1)
    cfa[Rloc] = img[Rloc,0]
    cfa[Gloc] = img[Gloc,1]
    cfa[Bloc] = img[Bloc,2]
    return cfa

# Four kernels to be convolved with CFA array
def _G_at_BR(cfa):
    kernel = [[ 0,   0, -1.0, 0,    0],
              [ 0,   0,    2, 0,    0],
              [-1.0, 2,    4, 2, -1.0],
              [ 0,   0,    2, 0,    0],
              [ 0,   0, -1.0, 0,    0]]
    return signal.convolve(cfa, kernel, mode='same')/8.

# red value at green pixel in red row and blue col or...
# blue value at green pixel in blue row and red col.
def _RB_at_G_in_RBrow_BRcol(cfa):
    kernel = [[ 0,  0, 0.5,  0,  0],
              [ 0, -1,   0, -1,  0],
              [-1,  4,   5,  4, -1],
              [ 0, -1,   0, -1,  0],
              [ 0,  0, 0.5,  0,  0]]
    return signal.convolve(cfa, kernel, mode='same')/8.

# red value at green pixel in blue row and red col or...
# blue value at green pixel in red row and blue col.
def _RB_at_G_in_BRrow_RBcol(cfa):
    kernel = [[ 0,  0, -1,  0,   0],
              [ 0, -1,  4, -1,   0],
              [0.5, 0,  5,  0, 0.5],
              [ 0, -1,  4, -1,   0],
              [ 0,  0, -1,  0,   0]]
    return signal.convolve(cfa, kernel, mode='same')/8.

# red value at blue pixel or...
# blue value at red pixel.
def _RB_at_BR(cfa):
    kernel = [[ 0,   0, -1.5, 0,    0],
              [ 0,   2,    0, 2,    0],
              [-1.5, 0,    6, 0, -1.5],
              [ 0,   2,    0, 2,    0],
              [ 0,   0, -1.5, 0,    0]]
    return signal.convolve(cfa, kernel, mode='same')/8.

def hqli(cfa):
    # initialize output arrays
    R = np.zeros_like(cfa, dtype=np.float64)
    G = np.zeros_like(cfa, dtype=np.float64)
    B = np.zeros_like(cfa, dtype=np.float64)
    # coordinate index arrays
    y, x = np.mgrid[0:cfa.shape[0], 0:cfa.shape[1]]

    # create groups of indices based on Bayer pattern
    Rloc = (np.mod(x, 2) == 0) & (np.mod(y, 2) == 0)
    Gloc = np.mod(x+y, 2) == 1
    Bloc = (np.mod(x, 2) == 1) & (np.mod(y, 2) == 1)
    G_in_Brow_Rcol = (np.mod(x, 2) == 0) & (np.mod(y, 2) == 1)
    G_in_Rrow_Bcol = (np.mod(x, 2) == 1) & (np.mod(y, 2) == 0)

    # copy data that doesn't need interpolation
    R[Rloc] = cfa[Rloc]
    G[Gloc] = cfa[Gloc]
    B[Bloc] = cfa[Bloc]

    # fill in the green data at the blue/red locations
    tmp = _G_at_BR(cfa)
    G[Rloc] = tmp[Rloc]
    G[Bloc] = tmp[Bloc]

    # fill in the blue/red data
    tmp = _RB_at_G_in_BRrow_RBcol(cfa)
    B[G_in_Rrow_Bcol] = tmp[G_in_Rrow_Bcol]
    R[G_in_Brow_Rcol] = tmp[G_in_Brow_Rcol]

    tmp = _RB_at_G_in_RBrow_BRcol(cfa)
    B[G_in_Brow_Rcol] = tmp[G_in_Brow_Rcol]
    R[G_in_Rrow_Bcol] = tmp[G_in_Rrow_Bcol]

    tmp = _RB_at_BR(cfa)
    B[Rloc] = tmp[Rloc]
    R[Bloc] = tmp[Bloc]

    R[R<0] = 0
    G[G<0] = 0
    B[B<0] = 0
    if cfa.dtype == np.uint8:
        R[R>255] = 255
        G[G>255] = 255
        B[B>255] = 255
    else:
        R[R>65535] = 65535
        G[G>65535] = 65535
        B[B>65535] = 65535

    return np.array(np.dstack([R, G, B]), dtype=cfa.dtype)

**Alleysons**

In [ ]:
def Alleysons(img):
    HG = np.array([(0,1,0), (1,4,1),(0,1,0)], dtype = float)/4
    HRB = np.array([(1,2,1), (2,4,2),(1,2,1)], dtype = float)/4
    [D,C,S]=img.shape
    mr=np.zeros((D,C))
    mb=np.zeros((D,C))
    for i in range(img.shape[0]):
      if i%2==0:
        for j in range(img.shape[1]):
          if j%2==0:
            mr[i,j]=1
    bgf=int(img.shape[0]-1)
    ef=int(img.shape[1]-1)
    mb[1:,1:]=mr[:bgf,:ef]
    mg=1-mr-mb
    Im=np.zeros_like(img)
    Im[:,:,0]=img[:,:,0]*mr
    Im[:,:,1]=img[:,:,1]*mg
    Im[:,:,2]=img[:,:,2]*mb
    FL = np.array([(-2,3,-6,3,-2),(3,4,2,4,3),(-6,2,48,2,-6),(3,4,2,4,3),(-2,3,-6,3,-2)], dtype = float)/64
    L = cv2.filter2D(Im, -1, FL)
    C=Im-L
    C1=C[:,:,0]*mr
    C2=C[:,:,1]*mg
    C3=C[:,:,2]*mb
    c1 = cv2.filter2D(C1, -1, HRB)
    c2 = cv2.filter2D(C2, -1, HG)
    c3 = cv2.filter2D(C3, -1, HRB)
    Ir=np.zeros(img.shape)
    Ir[:,:,0]=c1+L[:,:,0]
    Ir[:,:,1]=c2+L[:,:,1]
    Ir[:,:,2]=c3+L[:,:,2]
    return Ir


**bilinear interpolation**

In [ ]:
def Bili_inter(img):
    HG = np.array([(0,1,0), (1,4,1),(0,1,0)], dtype = float)/4
    HRB = np.array([(1,2,1), (2,4,2),(1,2,1)], dtype = float)/4
    [D,C,S]=img.shape
    mr=np.zeros((D,C))
    mb=np.zeros((D,C))
    for i in range(img.shape[0]):
      if i%2==0:
        for j in range(img.shape[1]):
          if j%2==0:
            mr[i,j]=1
    bgf=int(img.shape[0]-1)
    ef=int(img.shape[1]-1)
    mb[1:,1:]=mr[:bgf,:ef]
    mg=1-mr-mb
    Im=np.zeros_like(img)
    Im[:,:,0]=img[:,:,0]*mr
    Im[:,:,1]=img[:,:,1]*mg
    Im[:,:,2]=img[:,:,2]*mb
    Y1 = cv2.filter2D(Im[:,:,0], -1, HRB)
    Y2 = cv2.filter2D(Im[:,:,1], -1, HG)
    Y3 = cv2.filter2D(Im[:,:,2], -1, HRB)
    Im1=np.zeros_like(img)
    Im1[:,:,0]=Y1
    Im1[:,:,1]=Y2
    Im1[:,:,2]=Y3
    return Im1

In [ ]:
def show_im(img, title):
    figsize = (10, 10)
    plt.figure(figsize=figsize)
    plt.imshow(img)
    plt.title(title)
    plt.xticks([])
    plt.yticks([])
    plt.show()

def dis_img(root):
    fnames=os.listdir(root)
    Kodak_img=[]
    kodak_img=[]
    for i in range (24):
      kodak_img=np.uint8(Image.open(os.path.join(root,fnames[i])))
      Kodak_img.append(kodak_img)
    return Kodak_img

def get_PSNR(img,img2,i):
    print(skimage.metrics.peak_signal_noise_ratio(img, img2),"PSNR anh"+str(i))

def get_SSIM(img,img2,i):
    im1 = img
    im2=img2
    # Add an outer batch for each image.
    im1 = tf.expand_dims(im1, axis=0)
    im2 = tf.expand_dims(im2, axis=0)
    # Compute SSIM over tf.uint8 Tensors.
    ssim = tf.image.ssim(im1, im2, max_val=255, filter_size=11)
    print(ssim,"ssim"+str(i))

**import data**

In [ ]:
root='//content//gdrive//MyDrive//xử lý ảnh//giữa kỳ xla//kodak database'
Kodak_img=dis_img(root)

**PSNR**

In [ ]:
for i in range(24):
  img1=Bili_inter(Kodak_img[i])
  get_PSNR(Kodak_img[i],img1,i)

In [ ]:
for i in range(24):
  img1=Alleysons(Kodak_img[i])
  get_PSNR(Kodak_img[i],img1,i)

In [ ]:
for i in range(24):
    cfa_img = make_cfa(Kodak_img[i])
    img1 =hqli(cfa_img)
    get_PSNR(Kodak_img[i],img1,i)

**SSIM**

In [ ]:
for i in range (24):
  im1 = Kodak_img[i]
  im2 = Bili_inter(Kodak_img[i]).astype('uint8')
  # Add an outer batch for each image.
  im1 = tf.expand_dims(im1, axis=0)
  im2 = tf.expand_dims(im2, axis=0)
  # Compute SSIM over tf.uint8 Tensors.
  get_SSIM(im1,im2,i)


In [ ]:
for i in range (24):
  im1 = Kodak_img[i]
  im2 = Alleysons(Kodak_img[i]).astype('uint8')
  # Add an outer batch for each image.
  im1 = tf.expand_dims(im1, axis=0)
  im2 = tf.expand_dims(im2, axis=0)
  # Compute SSIM over tf.uint8 Tensors.
  get_SSIM(im1,im2,i)

In [ ]:
for i in range (24):
  im1 = Kodak_img[i]
  cfa_img = make_cfa(im1)
  im2 = hqli(cfa_img).astype('uint8')
  # Add an outer batch for each image.
  im1 = tf.expand_dims(im1, axis=0)
  im2 = tf.expand_dims(im2, axis=0)
  # Compute SSIM over tf.uint8 Tensors.
  get_SSIM(im1,im2,i)

In [ ]:
fig , axs = plt.subplots(nrows=1, n cols=4)
axs = axs.flatten()
axs[0].imshow(Kodak_img[1])
axs[0].axis('off')
axs[0].set_title("ảnh ban đầu ")
axs[1].imshow(Alleysons(Kodak_img[1])
axs[1].axis('off')
axs[1].set_title("ảnh ban đầu ")